#Configuraciones


In [0]:
from pyspark.sql.functions import *;
from pyspark.sql.types import *;
from array import array;

catalogo = "INAH"

db_plata = "INAH_PLATA" 
db_oro = "INAH_ORO"

In [0]:
#spark.sql("CREATE CATALOG IF NOT EXISTS {}".format(catalogo))

In [0]:
spark.sql("USE CATALOG {}".format(catalogo))
spark.sql("CREATE DATABASE IF NOT EXISTS {}".format(db_plata))
spark.sql("CREATE DATABASE IF NOT EXISTS {}".format(db_oro))

In [0]:
# Crear Volumen para almacenar los datos
spark.sql("USE {}".format(db_plata))
spark.sql("CREATE VOLUME IF NOT EXISTS datos_fuente_inah")

In [0]:

ruta_fuente= f"/Volumes/pieroocampo/{db_plata}/datos_fuente_inah/inah_visitantes/"
#ruta_db=f"abfss://datosfuente@{adls_name}.dfs.core.windows.net/DB"
# rutaplata = f"{ruta_db}/{db_plata}/inahdetalle"
# rutaoro = f"{ruta_db}/{db_oro}"
# spark.conf.set(f"fs.azure.account.auth.type.{adls_name}.dfs.core.windows.net", "SAS")
# spark.conf.set(f"fs.azure.sas.token.provider.type.{adls_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
# spark.conf.set(f"fs.azure.sas.fixed.token.{adls_name}.dfs.core.windows.net",sas)

#Accediendo al lakehouse

In [0]:
display(dbutils.fs.ls(ruta_fuente))

In [0]:
dbutils.fs.head(f"{ruta_fuente}inah_visitantes_detallado_anual_2020.csv")

In [0]:
dfDatosCrudos = ( spark.read
                 .format("csv")
                 .option("header","true")
                 .option("inferSchema","true")
                 .load(ruta_fuente)

)

In [0]:
display(dfDatosCrudos.limit(30))


#Arquitectura de medallón
La arquitectura de medallón se refiere a la etapa de la calidad de datos:
* **Bronce**: Son los datos crudos tal y como llegan de origen.
* **Plata**: Son los datos refindos para el primer consumo y consulta. Se alimenta de etapa Bronce
* **Oro**: Son los modelos de datos orientados a negocio e inteligencia de negocio. Se alimentan de etapa Plata
    

##Plata

In [0]:

esquema = (StructType([StructField("Estado",StringType(),True), 
     StructField("Clave_SIINAH",StringType(),True), 
     StructField("Siglas",StringType(),True),
     StructField("Centro_de_trabajo",StringType(),True),
     StructField("Anio",StringType(),True), 
     StructField("Mes",StringType(),True),
     StructField("Tipo_Visitantes",StringType(),True), 
     StructField("Numero_visitas",IntegerType(),True),
     StructField("Nacionalidad",StringType(),True)])
)

In [0]:
dfLimpieza =(spark.read.format("csv")
 .option("header","true")
 .schema(esquema)
 .option("encoding", "ISO-8859-1") #ANSI
 .load(ruta_fuente)
 )

display(dfLimpieza.limit(30))

In [0]:

dfLimpieza.write.mode("overwrite").saveAsTable(f"{catalogo}.{db_plata}.inah_detalle")

In [0]:
display(spark.sql(f"SELECT * FROM {catalogo}.{db_plata}.inah_detalle"))

##Oro

In [0]:
dfPlata= spark.table(f"{catalogo}.{db_plata}.inah_detalle")

###Inteligencia de negocio


In [0]:
(dfPlata.groupBy("Estado")
    .sum("Numero_visitas")
    .withColumnRenamed("sum(Numero_visitas)","Visitas")
    .write
    .mode("overwrite")
    .saveAsTable(f"{catalogo}.{db_oro}.ConteoEstado")
)

In [0]:
display(spark.sql(f"SELECT * FROM {catalogo}.{db_oro}.ConteoEstado"))

In [0]:
(
        dfPlata.groupBy("Centro_de_trabajo").sum("Numero_visitas").withColumnRenamed("sum(Numero_visitas)","Visitas")
        .write
        .mode("overwrite")
        .saveAsTable(f"{catalogo}.{db_oro}.ConteoCentroTrabajo")
)

In [0]:
display(spark.sql(f"SELECT * FROM {catalogo}.{db_oro}.ConteoCentroTrabajo"))

In [0]:
(
    dfPlata.groupBy("Nacionalidad").sum("Numero_visitas").withColumnRenamed("sum(Numero_visitas)","Visitas")
        .write
        .mode("overwrite")
        .saveAsTable(f"{catalogo}.{db_oro}.ConteoNacionalidad") 
)

In [0]:
display(spark.sql(f"SELECT * FROM {catalogo}.{db_oro}.ConteoNacionalidad"))

In [0]:
(
    dfPlata.groupBy("Anio","Mes").sum("Numero_visitas").orderBy("Anio").withColumnRenamed("sum(Numero_visitas)","Visitas")
        .write
        .mode("overwrite")
        .saveAsTable(f"{catalogo}.{db_oro}.ConteoAnioMes")
)

In [0]:
display(spark.sql(f"SELECT * FROM {catalogo}.{db_oro}.ConteoAnioMes"))